In [ ]:
## Data loading 

from argoverse.data_loading.argoverse_forecasting_loader import ArgoverseForecastingLoader

##set root_dir to the correct path to your dataset folder
root_dir = '/Users/xinyufang/Library/CloudStorage/Box-Box/Ferrari_SpA/crat-pred/dataset/argoverse/test_obs/data'

afl = ArgoverseForecastingLoader(root_dir)

print('Total number of sequences:',len(afl))

In [ ]:
print(afl[0])

In [ ]:
argoverse_forecasting_data = afl[0]
print(argoverse_forecasting_data.track_id_list)

In [ ]:
## Visualize

from argoverse.visualization.visualize_sequences import viz_sequence
seq_path = f"{root_dir}/1.csv"
viz_sequence(afl.get(seq_path).seq_df, show=True)
seq_path = f"{root_dir}/2.csv"
viz_sequence(afl.get(seq_path).seq_df, show=True)

In [4]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

with open('/Users/xinyufang/Library/CloudStorage/Box-Box/Ferrari_SpA/crat-pred/result_data/prediction.pkl', 'rb') as file:
    predictions = pickle.load(file)

with open('/Users/xinyufang/Library/CloudStorage/Box-Box/Ferrari_SpA/crat-pred/result_data/gts.pkl', 'rb') as file:
    gts = pickle.load(file)
# 
# with open('/Users/xinyufang/Library/CloudStorage/Box-Box/Ferrari_SpA/crat-pred/result_data/saved_pred_idx.pkl', 'rb') as file:
#     min_idx = pickle.load(file)  # For each key, choose the list of predictions with lowest fde value.

In [61]:
from argoverse.map_representation.map_api import ArgoverseMap
from argoverse.visualization.visualize_sequences import viz_sequence
avm = ArgoverseMap()

def plot_seq(obs_traj_abs, 
             pred_traj_fake_abs, 
             argo_id: int, 
             num_guess: int, 
             time_horizon: int, 
             show: bool = True, 
             plot_centerline: bool = True,
             save_plot: bool = False):
    
    obsv_color = "#ECA154"
    hist_color = "#d33e4c"
    pred_color = "#007672"
    
    data_val = '/Users/xinyufang/Library/CloudStorage/Box-Box/Ferrari_SpA/crat-pred/dataset/argoverse/val/data/'+str(argo_id)+'.csv'
    df_val = pd.read_csv(data_val)
    #df_val['TIMESTAMP'] = df_val['TIMESTAMP'].astype(str)
    
    target_value = obs_traj_abs[0,0]
    tolerance = 0.0001
    condition = (df_val['X'] >= (target_value - tolerance)) & (df_val['X'] <= (target_value + tolerance))
    track_id = df_val[condition]['TRACK_ID'].iloc[0]
    curr_time = df_val[condition]['TIMESTAMP'].iloc[0].round(5)
    print(f"Current Time: ", {curr_time})
    
    city = df_val['CITY_NAME'][0]
    seq_lane_props = avm.city_lane_centerlines_dict[city]
    
    df_hist_val = df_val[df_val['TRACK_ID'] == track_id]
    # Old filtering method
    #pred_traj_hist = df_hist_val[(df_hist_val['X'] < obs_traj_abs[0,0]) & (df_hist_val['Y'] < obs_traj_abs[0,1])][['X', 'Y']].to_numpy() 
    
    # New filtering method
    pred_traj_hist = df_hist_val[df_hist_val['TIMESTAMP'] <= curr_time][['X', 'Y']].to_numpy()

    
    plt.figure(0, figsize=(5, 5),dpi=100)
    
    if plot_centerline:
        x_min = np.min(np.concatenate((obs_traj_abs[:,0], pred_traj_hist[:,0], pred_traj_fake_abs[:,0]),axis=0))
        x_max = np.max(np.concatenate((obs_traj_abs[:,0], pred_traj_hist[:,0], pred_traj_fake_abs[:,0]),axis=0))
        y_min = np.min(np.concatenate((obs_traj_abs[:,1], pred_traj_hist[:,1], pred_traj_fake_abs[:,1]),axis=0))
        y_max = np.max(np.concatenate((obs_traj_abs[:,1], pred_traj_hist[:,1], pred_traj_fake_abs[:,1]),axis=0))
        
        lane_centerlines = []
        
        # Get lane centerlines which lie within the range of trajectories
        for lane_id, lane_props in seq_lane_props.items():
            lane_cl = lane_props.centerline
    
            if np.min(lane_cl[:, 0]) < x_max and np.min(lane_cl[:, 1]) < y_max and np.max(lane_cl[:, 0]) > x_min and np.max(lane_cl[:, 1]) > y_min:
                lane_centerlines.append(lane_cl)
    
        for lane_cl in lane_centerlines:
            plt.plot(lane_cl[:, 0], lane_cl[:, 1], "--", color="grey", alpha=1, linewidth=1, zorder=0)
            plt.xlim([x_min - 10, x_max + 10])
            plt.ylim([y_min - 10, y_max + 10])
    
    # Plot observed trajectory and its end point
    plt.plot(obs_traj_abs[:,0], obs_traj_abs[:,1], color=obsv_color, label="Observed", alpha=1, linewidth=3, zorder=15)
    plt.plot(obs_traj_abs[-1,0], obs_traj_abs[-1,1], "o", color=obsv_color, markersize=9, zorder=15)
    
    # Plot history trajectory and its end point
    plt.plot(pred_traj_hist[:,0], pred_traj_hist[:,1], color=hist_color, label="History", alpha=0.8, linewidth=3, zorder=20)
    #plt.plot(pred_traj_hist[-1,0], pred_traj_hist[-1,1], "o", color=hist_color, markersize=9, zorder=20)
    
    # Plot predicted trajectory and its end point
    plt.plot(pred_traj_fake_abs[:,0], pred_traj_fake_abs[:,1], color=pred_color, label="Prediction", alpha=1, linewidth=3, zorder=15)
    plt.plot(pred_traj_fake_abs[-1,0], pred_traj_fake_abs[-1,1], "o", color=pred_color, markersize=9, zorder=15)
    
    print(f'Vehicle ID: {track_id}')
    plt.title('File ID: '+str(argo_id)+'; '+'Guess: '+str(num_guess)+'; '+'Horizon: '+str(time_horizon))
    plt.legend()
    
    #print(f'History Trajectory: {pred_traj_hist}')
    #print(f'Prediction Trajectory: {pred_traj_fake_abs}')
    
    if save_plot:
        plt.savefig('/Users/xinyufang/Library/CloudStorage/Box-Box/Ferrari_SpA/crat-pred/result_data/plots/'+str(argo_id)+'_'+str(num_guess)+'_'+str(time_horizon)+'.png',dpi=300)
    
    if show:
        plt.show() 
    else:
        plt.close()
        


In [62]:
import itertools

max_guesses = 1
forecasted_probabilities = None
horizon = 30
num_plot = 100

# for k, v in gts.items():
for k, v in itertools.islice(gts.items(), num_plot):  # k is same as 'argo_id = int(Path(filename).stem)'
    curr_min_ade = float("inf")
    curr_min_fde = float("inf")
    min_idx = 0
    max_num_traj = min(max_guesses, len(predictions[k]))

    # If probabilities available, use the most likely trajectories, else use the first few
    if forecasted_probabilities is not None:
        sorted_idx = np.argsort([-x for x in forecasted_probabilities[k]], kind="stable")
        # sorted_idx = np.argsort(forecasted_probabilities[k])[::-1]
        pruned_probabilities = [forecasted_probabilities[k][t] for t in sorted_idx[:max_num_traj]]
        # Normalize
        prob_sum = sum(pruned_probabilities)
        pruned_probabilities = [p / prob_sum for p in pruned_probabilities]
    else:
        sorted_idx = np.arange(len(predictions[k]))
    pruned_trajectories = [predictions[k][t] for t in sorted_idx[:max_num_traj]]

    for j in range(len(pruned_trajectories)): #j is from 0 to 6 for max_guesses
        x_pred = pruned_trajectories[j][:horizon][:,0]
        y_pred = pruned_trajectories[j][:horizon][:,1]
        x_gts = v[:horizon][:,0].numpy()
        y_gts = v[:horizon][:,1].numpy()
        traj_pred = pruned_trajectories[j][:horizon]
        traj_gts = v[:horizon].numpy()
        #if k == 10001 and j == 0: 
        plot_seq(traj_gts,traj_pred,k,j,horizon,show=False,plot_centerline=True,save_plot=True)


Current Time:  {315990610.45172}
Vehicle ID: 00000000-0000-0000-0000-000000189661
Current Time:  {315970250.73265}
Vehicle ID: 00000000-0000-0000-0000-000000030539
Current Time:  {315971864.42537}
Vehicle ID: 00000000-0000-0000-0000-000000077098
Current Time:  {315972903.22938}
Vehicle ID: 00000000-0000-0000-0000-000000062611
Current Time:  {315968318.43384}
Vehicle ID: 00000000-0000-0000-0000-000000008375
Current Time:  {315978474.84656}
Vehicle ID: 00000000-0000-0000-0000-000000087889
Current Time:  {315967100.83683}
Vehicle ID: 00000000-0000-0000-0000-000000004628
Current Time:  {315973141.30457}
Vehicle ID: 00000000-0000-0000-0000-000000087237
Current Time:  {315967298.73895}
Vehicle ID: 00000000-0000-0000-0000-000000003325
Current Time:  {315977747.16166}
Vehicle ID: 00000000-0000-0000-0000-000000106507
Current Time:  {315976491.96474}
Vehicle ID: 00000000-0000-0000-0000-000000129525
Current Time:  {315968536.07056}
Vehicle ID: 00000000-0000-0000-0000-000000016832
Current Time:  {

In [ ]:
# Preprocess gts and predictions

flattened_gts = []
flattened_pred = []

print('Check issues with preprocess:')

print(f'Size of gts is: {len(gts)}')
print(f'Size of predictions is: {len(predictions)}')

for key in gts:
    for array in gts[key]:
        # Append each pair as a tuple
        flattened_gts.append((array[0], array[1])) 

print(flattened_gts[1])

for key in predictions:
    for array in predictions[key][0]:  # 0 denotes lowest fde for each list of predictions in each key 
        # Append each pair as a tuple
        flattened_pred.append((array[0], array[1]))

# Split the tuples into two lists: one for 'X' and one for 'Y'
X_data_gts = [item[0].numpy() for item in flattened_gts]
Y_data_gts = [item[1].numpy() for item in flattened_gts]

print(X_data_gts[1])

X_data_pred = [item[0] for item in flattened_pred]
Y_data_pred = [item[1] for item in flattened_pred]

# Create the new dictionary for gts
gts_pred = {'X': X_data_gts, 'Y': Y_data_gts, 'X_PRED': X_data_pred, 'Y_PRED': Y_data_pred}

print(gts_pred['X'][0])
print(gts_pred['Y'][0])
print(gts_pred['X_PRED'][0])
print(gts_pred['Y_PRED'][0])

